In [ ]:
import numpy as np
import copy


import scipy.special
import torch
import torch.nn as nn

In [ ]:
# TODO: Convert the notebook to /tests directory
n_ops = 7
n_nodes = 3

alpha_normal = []
alpha_reduce = []

for i in range(n_nodes):
    # create alpha parameters over parallel operations
    alpha_normal.append(nn.Parameter(
        1e-3 * torch.randn(i + 2, n_ops)))
    alpha_reduce.append(nn.Parameter(
        1e-3 * torch.randn(i + 2, n_ops)))
    
    
[i.shape for i in alpha_normal]

In [ ]:
# Set alphas to zero

In [ ]:
alphas = []
alpha_reduce = []
n_ops = 7

class A(nn.Module):
    def __init__(self):
        super(A, self).__init__()
        self.alphas = []
        for i in range(n_nodes):
            # create alpha parameters over parallel operations
            self.alphas.append(nn.Parameter(
                1e-3 * torch.randn(i + 2, n_ops)))
            alpha_reduce.append(nn.Parameter(
                1e-3 * torch.randn(i + 2, n_ops)))

# alphas = [torch.tensor(np.random.normal(0, 0.4, size=(2,7))),
#          torch.tensor(np.random.normal(0, 0.4, size=(3,7))), 
#          torch.tensor(np.random.normal(0, 0.4, size=(4,7)))]


# drop = np.array(weights[i, :]).argsort()[:ops_drop]
num_to_drop = [2, 2, 0]
A = A()
# add stages loop
# TODO: Currently the 0 values are sometimes higher than negative values that might show up
for stage in range(3):
    n_ops_reduce = 5 - num_to_drop[stage]

    for alpha in A.alphas:
        topk, indices = torch.topk(alpha[:, :], n_ops_reduce)

        mask = torch.zeros(len(alpha), n_ops)
        a_min = torch.min(alpha)
        epsilon = 0.0001
        print(alpha)
        print(indices)
#         print(mask)
#         print(indices)
#         print(mask.scatter_(1, indices, True) * alpha)
        alpha.data = mask.scatter_(1, indices, True) * alpha
        alpha.data[alpha.data == 0] = a_min - epsilon
        print(alpha)
        break
    break
        
#     print(alphas, "\n")
        
# A.alphas

In [ ]:
# Test for skip-connections

In [ ]:
PRIMITIVES = [
    "max_pool_3x3",
    "avg_pool_3x3",
    "skip_connect",  # identity
    "conv_1x5_5x1",
    "conv_3x3",
    "sep_conv_3x3",
    "dil_conv_3x3"
]

alpha = [torch.tensor(np.random.normal(0, 0.4, size=(2,7))),
         torch.tensor(np.random.normal(0, 0.4, size=(3,7))), 
         torch.tensor(np.random.normal(0, 0.4, size=(4,7)))]

alpha_pairwise = [torch.tensor([1]), 
                  torch.tensor([0.3346, 0., 0.]), 
                  torch.tensor([0.0, 0.0, 0.0, 0.1686, 0.0, 0.0])]

In [ ]:
def convert_tensor_alphas(alpha_concat, nodes=3):
    alphas = []
    for a_i in get_edge_indices(nodes):
        alphas.append(
            torch.Tensor(alpha_concat[a_i[0]:a_i[1]]))
        
    # print(alpha_concat, alphas)
    return alphas

def get_edge_indices(nodes=3):
    # Amount of nodes for each edge
    j = [i for i in range(2, nodes+2)] 
    
    prev = 0
    indices = []
    for i in j:
        if prev != 0:
            indices.append((sum(j[:j.index(prev)+1]), 
                            sum(j[:j.index(i)+1])))
        else:
            indices.append((0, sum(j[:j.index(i)+1])))
        prev = i
    return indices

def parse(alpha, k, primitives=PRIMITIVES):
    gene = []
    for edges in alpha:
        # edges: Tensor(n_edges, n_ops)
        edge_max, primitive_indices = torch.topk(
            edges[:, :], 1
        )  # ignore 'none' ##removed none
        topk_edge_values, topk_edge_indices = torch.topk(edge_max.view(-1), k)
        node_gene = []
        for edge_idx in topk_edge_indices:
            prim_idx = primitive_indices[edge_idx]
            prim = primitives[prim_idx]
            node_gene.append((prim, edge_idx.item()))

        gene.append(node_gene)
    return gene


parse(alpha, 2, switches_normal)

In [ ]:
import torch.nn.functional as F


weights_normal = [F.softmax(a).data.cpu().numpy() for a in alpha]
weights_normal = np.concatenate(weights_normal, axis=0)
switches = switches_normal

idxs = np.where(switches[i])[0].tolist()
drop = np.array(weights_normal[i, :]).argsort()[:2]


idxs, drop, weights_normal[i]


In [ ]:
get_edge_indices()

In [ ]:
alpha

[a.shape for a in alpha]

In [ ]:
nodes = 3

def get_edge_indices(nodes=3):
    # Amount of nodes for each edge
    j = [i for i in range(2, nodes+2)] 
    
    prev = 0
    indices = []
    for i in j:
        if prev != 0:
            indices.append((sum(j[:j.index(prev)+1]), 
                            sum(j[:j.index(i)+1])))
        else:
            indices.append((0, sum(j[:j.index(i)+1])))
        prev = i
    return indices

def find_indice(row_idx, nodes=3):
    edges = get_edge_indices(nodes)

    for i, (lower, upper) in enumerate(edges):
        if row_idx >= lower and row_idx <= upper:
            # return index
            idx_alpha = 0 if row_idx-lower == 0 else row_idx-lower-1
            return i, idx_alpha
        
# alpha[][]
for i in range(0, 10):
    print(find_indice(i))

In [ ]:

alpha = [torch.tensor(np.random.normal(0, 0.4, size=(2,7))).cuda(),
         torch.tensor(np.random.normal(0, 0.4, size=(3,7))).cuda(), 
         torch.tensor(np.random.normal(0, 0.4, size=(4,7))).cuda()]
alpha[0][0][2] = 1
alpha[0][1][2] = 1
alpha[1][0][2] = 1
alpha[1][1][2] = 1
alpha[2][0][2] = 1
alpha[2][1][2] = 1

print(np.concatenate(
    [a.detach().cpu().numpy() for a in alpha],
    axis=0))

In [ ]:
def get_edge_indices(nodes=3):
    # Amount of nodes for each edge
    j = [i for i in range(2, nodes+2)] 
    
    prev = 0
    indices = []
    for i in j:
        if prev != 0:
            indices.append((sum(j[:j.index(prev)+1]), 
                            sum(j[:j.index(i)+1])))
        else:
            indices.append((0, sum(j[:j.index(i)+1])))
        prev = i
    return indices

def find_indice(row_idx, nodes=3):
    edges = get_edge_indices(nodes)

    for i, (lower, upper) in enumerate(edges):
        if row_idx >= lower and row_idx <= upper:
            # return index
            idx_alpha = 0 if row_idx-lower == 0 else row_idx-lower-1
            return i, idx_alpha

def limit_skip_connections_alphas(alpha, primitives, k=2, nodes=3, num_of_skip_connections=2):
    """Mutate the alpha in-place and return the corresponding gene"""
    gene = parse(alpha, k=k)
    num_sk_enabled = sum([1 for edge in gene
                          for op in edge if op[0] == "skip_connect"])
    epsilon = 0.00
    
    if num_sk_enabled < num_of_skip_connections:
        return gene
    else:
        sk_idx = primitives.index("skip_connect")
        alphas_concat = np.concatenate(
            [a.detach().numpy() for a in alpha],
            axis=0)
        sk_alphas = alphas_concat[:, sk_idx-1:sk_idx]

        for i in range(len(sk_alphas)):
            # Pick skip-connection index with lowest alpha value
            row_idx = np.argmin(sk_alphas)
            # Set to inf so we don't pick this again
            sk_alphas[row_idx] = float("inf")
            
            edge_idx, row_idx = find_indice(row_idx)
            print(alpha[edge_idx][row_idx])
            print(torch.min(alpha[edge_idx][row_idx]))

            alpha[edge_idx][row_idx][sk_idx] = torch.min(alpha[edge_idx][row_idx]) - epsilon
            gene = parse(alpha, k=k)
            
            num_sk_enabled = sum([1 for edge in gene
                                  for op in edge if op[0] == "skip_connect"])
            if num_sk_enabled < num_of_skip_connections:
                return gene

limit_skip_connections_alphas(alpha, PRIMITIVES)
print(np.concatenate(
    [a.detach().numpy() for a in alpha],
    axis=0))


In [ ]:

def reduce_skip_connections(alphas, num_of_skip_connections=2, nodes=3, 
                           k=2, primitives=PRIMITIVES):
    sk_idx = primitives.index("skip_connect")
    alphas = [a.detach().numpy() for a in alphas]
    alpha_concat = np.concatenate(alphas, axis=0)
    
    # Number of skip-connections enabled in switches
    gene = parse(alphas, switches, k=k)
    num_sk_enabled = sum([1 for edge in gene 
                          for op in edge if op[0] == "skip_connect"])
    
    sk_a = alpha_concat[:, sk_idx-1:sk_idx]
    if num_sk_enabled < num_of_skip_connections:
        alphas = convert_tensor_alphas(alpha_concat)
        gene = parse(alphas, switches, k=k)
        return gene
    else:
        it = 0
        while num_sk_enabled > num_of_skip_connections:
            print("########## iteration", it)
            it += 1
            # Pick skip-connection index with lowest alpha 
            # value
            row_idx = np.argmin(sk_a)
            sk_a[row_idx] = float("inf")

        #     # set alphas to -inf to make sure, prevent it from
        #     # being picked. 
            alpha_concat[row_idx][sk_idx] = float("-inf")
            alphas = convert_tensor_alphas(alpha_concat)

            gene = parse(alphas, switches, k=k)
            num_sk_enabled = sum([1 for edge in gene 
                                  for op in edge if op[0] == "skip_connect"])

            if num_sk_enabled <= num_of_skip_connections:
                # return the new switches
                return gene

limit_skip_connections(alpha)

In [ ]:

alphas = [torch.tensor(np.random.normal(0, 0.4, size=(2,3))).cuda(),
         torch.tensor(np.random.normal(0, 0.4, size=(3,3))).cuda(), 
         torch.tensor(np.random.normal(0, 0.4, size=(4,3))).cuda()]

alphas = [a.detach().cpu().numpy() for a in alphas]
alpha_concat = np.concatenate(alphas, axis=0)
alpha_concat

In [ ]:
sk_a = alpha_concat[:, 1:2]

np.argmin(sk_a)

it = 0
while num_sk_enabled > num_of_sk:
    print("########## iteration", it)
    it += 1
    # Pick skip-connection index with lowest alpha 
    # value
    row_idx = np.argmin(sk_a)
    sk_a[row_idx] = float("inf")

#     # set alphas to -inf to make sure, prevent it from
#     # being picked. 
    alpha_concat[row_idx][sk_idx] = float("-inf")
    alphas = convert_tensor_alphas(alpha_concat)

    gene = parse(alphas, switches, k=k)
    num_sk_enabled = sum([1 for edge in gene 
                          for op in edge if op[0] == "skip_connect"])

    if num_sk_enabled <= num_of_sk:
        # return the new switches
        return gene